# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-1278000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'J.J82.A98632.T52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "1H", "1N", "2S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 1, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '3D', 'insta_score': 0.467, 'expected_score': -1, 'adjustment': 23},
 {'call': 'PASS',
  'insta_score': 0.391,
  'expected_score': -33,
  'adjustment': 20}]

In [4]:
bid.samples

['J.J8x.A98xxx.Txx ATx.A9xxx.Tx.Axx Kx.KQxx.KJxx.K98 Q98xxxx.T.Q.QJxx 0.83908 \n 1H-1N-2S-3D-P-P-P (8.5, 8.8)  \n 1H-1N-2S-P-3S-P-P-P (9.9, 9.9) ',
 'J.J8x.A98xxx.Txx Txxx.ATxxx.x.AQJ A8.K9xx.KQx.Kxxx KQ9xxx.Q.JTx.98x 0.83853 \n 1H-1N-2S-3D-P-P-P (8.8, 9.2)  \n 1H-1N-2S-P-3S-P-P-P (9.6, 10.0) ',
 'J.J8x.A98xxx.Txx Ax.AKTxx.Txxx.xx KTx.Qxxx.KQ.KQ9x Q98xxxx.9.J.AJ8x 0.82745 \n 1H-1N-2S-3D-P-P-3S-P-P-P (10.0, 9.8)  \n 1H-1N-2S-P-3H-P-4S-P-P-P (10.0, 9.8) ',
 'J.J8x.A98xxx.Txx Tx.KQT9x.Kx.QJxx AQ8.Axxx.Qx.A98x K9xxxxx.x.JTx.Kx 0.81989 \n 1H-1N-2S-3D-P-P-3S-P-P-P (8.0, 7.9)  \n 1H-1N-2S-P-3H-P-4S-P-P-P (8.0, 7.9) ',
 'J.J8x.A98xxx.Txx Axx.AKT9xx.x.J9x Kx.Qxx.KQJT.KQxx QT98xxx.x.xx.A8x 0.81703 \n 1H-1N-2S-3D-3S-P-P-P (10.0, 10.5)  \n 1H-1N-2S-P-3S-P-P-P (10.0, 10.5) ',
 'J.J8x.A98xxx.Txx 98.AKT9xx.KTx.8x ATxx.Qxx.QJ.AK9x KQxxxx.x.xx.QJxx 0.81236 \n 1H-1N-2S-3D-P-P-P (8.3, 8.8)  \n 1H-1N-2S-P-3H-P-P-P (6.5, 6.8) ',
 'J.J8x.A98xxx.Txx Ax.ATxxx.Tx.AJ8x Qxx.KQ9x.KQxx.Kx KT98xxx.x.J.Q9xx 0.79413 

In [5]:
auction = ["PAD_START", "PAD_START", "1H", "1N", "2S","3D","3S","PASS","4S"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'X',
  'insta_score': 0.228,
  'expected_score': 1635,
  'adjustment': 114},
 {'call': 'PASS',
  'insta_score': 0.628,
  'expected_score': 339,
  'adjustment': 314},
 {'call': '5D',
  'insta_score': 0.143,
  'expected_score': -1935,
  'adjustment': 71}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [6]:
bid.samples

['J.J8x.A98xxx.Txx Q9xx.AKT9x.x.A9x Kxx.Qxx.KQJx.KQ8 AT8xx.xx.Tx.Jxxx 0.51589 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (3.2, 6.1)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (3.2, 6.1)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (3.8, 7.1) ',
 'J.J8x.A98xxx.Txx KQ8.AQxxxx.Tx.Jx Axx.KT9x.KQJ.Q8x T9xxxx..xx.AK9xx 0.51310 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (2.5, 6.0)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (2.5, 6.0)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (3.8, 8.1) ',
 'J.J8x.A98xxx.Txx 98xx.AK9xx.J.KJx KTx.QTxx.KQT.AQx AQxxx.x.xxx.98xx 0.50541 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (3.7, 7.2)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (3.7, 7.2)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (3.1, 7.3) ',
 'J.J8x.A98xxx.Txx QT8x.KQT9x.K.Axx A9x.Axx.Qxxx.KQ9 Kxxxx.xx.JT.J8xx 0.50285 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (4.6, 6.5)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (4.6, 6.5)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (3.0, 6.4) ',
 'J.J8x.A98xxx.Txx AK9.9xxxxx.T.KQ9 T8x.AKQT.KQJx.xx Qxxxxx..xx.AJ8xx 0.49880 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (1.2, 7.2)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (1.2, 7.2)

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: False
J.J8x.A98xxx.Txx 8xxx.Txx.KQx.J8x AKQxxx.AKx..AKxx T9.Q9xx.JTxx.Q9x
J.J8x.A98xxx.Txx Kxxx.xxx.Qxx.J8x AQTxxx.AKQ.K.Axx 98.T9xx.JTx.KQ9x
J.J8x.A98xxx.Txx Txx.Txx.QJx.K9xx AKQ9xxx.AKQ.K.xx 8x.9xxx.Txx.AQJ8
J.J8x.A98xxx.Txx 8xx.QT9.KJxx.J8x AKQxxx.AKx.Q.Axx T9x.xxxx.Tx.KQ9x
J.J8x.A98xxx.Txx 9xxx.K9x.QTx.Q8x AKQ8xx.AQxx..AJx Tx.Txx.KJxx.K9xx
J.J8x.A98xxx.Txx 9x.xxxx.Jxxx.Qxx AKQT8x.A9x.K.AK8 xxxx.KQT.QT.J9xx
J.J8x.A98xxx.Txx Axx.QT9x.Tx.J8xx KQ8xxx.Kx.Kx.AKQ T9x.Axxx.QJx.9xx
J.J8x.A98xxx.Txx xx.Qxxx.Qxx.Q8xx AKQTxxx.A9.J.AKJ 98x.KTxx.KTx.9xx
J.J8x.A98xxx.Txx Qx.9xxx.Txx.AQ8x AK8xxx.AK.Q.KJ9x T9xx.QTxx.KJx.xx
J.J8x.A98xxx.Txx xxx.K9xx.KJTx.Jx AKQT98.Axx.x.AKx xxx.QTx.Qx.Q98xx
